# Experiments with postgis

Here the goal is to check if we can use postgis as our geometrical/geographical indexing engine

In [1]:
import psycopg2 as psy
import pandas as pd

In [2]:
from osgeo import ogr
from osgeo import osr

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(3003)

transform = osr.CoordinateTransformation(source, target)

def map_to_monte_mario(wkt):
    geom = ogr.CreateGeometryFromWkt(wkt)
    geom.Transform(transform)
    return geom.ExportToWkt()

print(map_to_monte_mario('POINT(9.1092760 39.2284606)'))
POLY = "POLYGON ((9.147340 39.239940, 9.147380 39.239940, 9.147380 39.239960, 9.147340 39.239980, 9.147340 39.239940))"
print(map_to_monte_mario(POLY))

POINT (1509456.93227705 4342142.10053815)
POLYGON ((1512740.33316648 4343420.69829172,1512743.78517678 4343420.70389382,1512743.78157428 4343422.92345131,1512740.32596343 4343425.1374067,1512740.33316648 4343420.69829172))


In [3]:
# create testdb
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
con = psy.connect("dbname=postgres host=localhost user=postgres password=foobar")

with con:
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    with con.cursor() as cur:
        cur.execute("DROP DATABASE IF EXISTS testdbgis;")
        cur.execute("CREATE DATABASE testdbgis;")


In [4]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")

with con:
    with con.cursor() as cur:   
        cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
        cur.execute("SELECT postgis_version();")
        print(cur.fetchall())
        cur.execute("SELECT srid, auth_name, proj4text FROM spatial_ref_sys LIMIT 3;")
        for x in cur.fetchall():
            print(x)
        cur.execute("SELECT srid, auth_name, proj4text FROM spatial_ref_sys WHERE srid = 3003")
        print(cur.fetchall())
con.close()

[('2.5 USE_GEOS=1 USE_PROJ=1 USE_STATS=1',)]
(3819, 'EPSG', '+proj=longlat +ellps=bessel +towgs84=595.48,121.69,515.35,4.115,-2.9383,0.853,-3.408 +no_defs ')
(3821, 'EPSG', '+proj=longlat +ellps=aust_SA +no_defs ')
(3824, 'EPSG', '+proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs ')
[(3003, 'EPSG', '+proj=tmerc +lat_0=0 +lon_0=9 +k=0.9996 +x_0=1500000 +y_0=0 +ellps=intl +towgs84=-104.1,-49.1,-9.9,0.971,-2.917,0.714,-11.68 +units=m +no_defs ')]


## Saving point-probes location

Simplest scenario, we are just saving sensors positions.

In [5]:
sensors = pd.read_csv('./data/sensor_data_denorm.csv', index_col='id')

In [6]:
sensors.head(n=2)

,name,note,station_id,sensor_type,latitude,longitude,station_name
id,,,,,,,
1,S1,Viale Marconi ingresso Viale Marconi ingr dx,1,4,39.239953,9.147368,MARCONI_I
2,S2,Viale Marconi ingresso Viale Marconi ingr sx,1,4,39.239953,9.147368,MARCONI_I


In [7]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS sensors;
            CREATE TABLE sensors (
              id int4 primary key,
              name VARCHAR(20),
              note VARCHAR(100),
              station_id int4,
              sensor_type int4,
              geom geometry(POINT,3003) );
        """)
con.close()

In [8]:
SQL = """INSERT INTO sensors (id, name, note, station_id, sensor_type, geom)
         VALUES (%s, %s, %s, %s, %s, ST_GeomFromText(%s, 3003));"""

con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        for i, v in zip(sensors.index, sensors.values):
            (name, note, st_id, stype, lat, lon) = v[:-1]
            cur.execute(SQL, (i, name, 
                              note, st_id, stype, 
                              map_to_monte_mario("POINT (%s %s)" % (lon, lat))))
con.close()

['S1' 'Viale Marconi ingresso Viale Marconi ingr dx' 1 4
 39.239952771999995 9.14736801385 'MARCONI_I']
POINT (1512742.74846858 4343422.11962438)
['S2' 'Viale Marconi ingresso Viale Marconi ingr sx' 1 4
 39.239952771999995 9.14736801385 'MARCONI_I']
POINT (1512742.74846858 4343422.11962438)
['S3' 'Viale Marconi uscita Viale Marconi uscita dx' 2 4 39.2399100739
 9.14742141869 'MARCONI_U']
POINT (1512747.36501236 4343417.38856126)
['S4' 'Viale Marconi uscita Viale Marconi uscita sx' 2 4 39.2399100739
 9.14742141869 'MARCONI_U']
POINT (1512747.36501236 4343417.38856126)
['S5'
 'Circonvallazione Pirri - Via Quasimodo IN Circonvallazione Pirri - Via Quasimodo'
 3 4 39.2531666371 9.11308806904 'PIRRI_I']
POINT (1509782.55615189 4344884.32612236)
['S6'
 'Circonvallazione Pirri - Via Quasimodo IN Circonvallazione Pirri - Via Quasimodo'
 3 4 39.2531666371 9.11308806904 'PIRRI_I']
POINT (1509782.55615189 4344884.32612236)
['S7'
 'Circonvallazione Pirri - Via Quasimodo OUT Circonvallazione Pirri 

In [9]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
SQL = """
SELECT * 
FROM sensors 
WHERE ST_Contains(ST_GEOMFROMTEXT(%s, 3003), sensors.geom);
"""
RECT = "POLYGON((9.147340 39.239940, 9.147380 39.239940, 9.147380 39.239960, 9.147340 39.239980, 9.147340 39.239940))"

with con:
    with con.cursor() as cur:
        cur.execute(SQL, (map_to_monte_mario(RECT),))
        for x in cur.fetchall():
            print(x)

(1, 'S1', 'Viale Marconi ingresso Viale Marconi ingr dx', 1, 4, '0101000020BB0B000009A39BBF2615374104EDA7879F915041')
(2, 'S2', 'Viale Marconi ingresso Viale Marconi ingr sx', 1, 4, '0101000020BB0B000009A39BBF2615374104EDA7879F915041')


## Saving area-probes location

In [10]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS areaprobe;
            CREATE TABLE areaprobe (
              id int4 primary key,
              name VARCHAR(20),
              sensor_type int4,
              geom geometry(POLYGON,3003) );
        """)
con.close()

In [11]:
SQL = """INSERT INTO areaprobe (id, name, geom)
         VALUES (%s, %s, ST_GeomFromText(%s, 3003));"""
RECT = "POLYGON((8.7519483  39.5057942, 8.7519483  38.9511270, 9.4666037  39.5057942,9.4666037  38.9511270, 8.7519483  39.5057942))"


con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute(SQL, (12, 'Radar@UNICA', map_to_monte_mario(RECT)))
con.close()

In [12]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute("SELECT * from areaprobe")
        for x in cur.fetchall():
            print(x)
con.close()

(12, 'Radar@UNICA', None, '0103000020BB0B000001000000050000000CBF06162B90364191E8EB1C74AE5041CE41864C828F364171627E2D577250414A2BF8A12C803741B9FDF4AF86AE5041C443D66A69813741FCD677AD697250410CBF06162B90364191E8EB1C74AE5041')


## Generic geo objects

In [13]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")

with con:
    with con.cursor() as cur:
        cur.execute("""
            DROP TABLE IF EXISTS geothings;
            CREATE TABLE geothings (
              id int4 primary key,
              name VARCHAR(20),
              type int4,
              geom geometry);
        """)
con.close()

In [15]:
SQL = """INSERT INTO geothings (id, name, type, geom)
         VALUES (%s, %s, %s, ST_GeomFromText(%s, 3003));"""

con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        for i, v in zip(sensors.index, sensors.values):
            (name, note, st_id, stype, lat, lon) = v[:-1]
            cur.execute(SQL, (i, name, stype, 
                              map_to_monte_mario("POINT (%s %s)" % (lon, lat))))
            
con.close()

In [16]:
SQL = """INSERT INTO geothings (id, name, type, geom)
         VALUES (%s, %s, %s, ST_GeomFromText(%s, 3003));"""
RECT = "POLYGON((8.7519483  39.5057942, 8.7519483  38.9511270, 9.4666037  39.5057942,9.4666037  38.9511270, 8.7519483  39.5057942))"

con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute(SQL, (2000, 'Radar@UNICA', 4, map_to_monte_mario(RECT)))
con.close()

In [23]:
con = psy.connect("dbname=testdbgis host=localhost user=postgres password=foobar")
with con:
    with con.cursor() as cur:
        cur.execute("""
        SELECT name, ST_AsText(ST_Transform(geom,4326)) from geothings
        WHERE ST_DWithin(geom, ST_GeomFromText(%s, 3003), 1000);
        """, (map_to_monte_mario('POINT(9.1092760 39.2284606)'), ))
        for x in cur.fetchall():
            print(x)
con.close()

('S51', 'POINT(9.12039292999226 39.2286574388301)')
('S52', 'POINT(9.12039292999226 39.2286574388301)')
('S53', 'POINT(9.12039292999226 39.2286574388301)')
('S54', 'POINT(9.12039292999226 39.2286574388301)')
('S111', 'POINT(9.10695418035839 39.2365039319305)')
('S112', 'POINT(9.10695418035839 39.2365039319305)')
('S113', 'POINT(9.10701257418839 39.2365853430306)')
('S114', 'POINT(9.10701257418839 39.2365853430306)')
('S119', 'POINT(9.10457031481892 39.231571742031)')
('S120', 'POINT(9.10453740192893 39.231537201431)')
('S121', 'POINT(9.11249696609539 39.2277638517308)')
('S122', 'POINT(9.11247042344534 39.2277284869308)')
('S127', 'POINT(9.11583882280433 39.2307833517303)')
('S128', 'POINT(9.11583882280433 39.2307833517303)')
('S129', 'POINT(9.11592588268433 39.2308507887303)')
('S130', 'POINT(9.11592588268433 39.2308507887303)')
('S135', 'POINT(9.10244445178875 39.2218611124318)')
('S136', 'POINT(9.10244445178875 39.2218611124318)')
('S137', 'POINT(9.10236814166885 39.2218413722318)')